In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%run 0_parameters.ipynb

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload

This notebook introduces the problem addressed in this paper:

 - localizating an object in a large image
 - foveation
 - action (saccade)
       
      
{'w': 28, 'minibatch_size': 100, 'train_batch_size': 50000, 'test_batch_size': 10000, 'noise_batch_size': 1000, 'mean': 0.1307, 'std': 0.3081, 'what_offset_std': 15, 'what_offset_max': 25, 'N_pic': 128, 'offset_std': 30, 'offset_max': 34, 'noise': 0.75, 'contrast': 0.7, 'sf_0': 0.1, 'B_sf': 0.1, 'do_mask': True, 'N_theta': 6, 'N_azimuth': 24, 'N_eccentricity': 10, 'N_phase': 2, 'rho': 1.41, 'bias_deconv': True, 'p_dropout': 0.0, 'dim1': 1000, 'dim2': 1000, 'lr': 0.005, 'do_adam': True, 'bn1_bn_momentum': 0.5, 'bn2_bn_momentum': 0.5, 'momentum': 0.3, 'epochs': 60, 'num_processes': 1, 'no_cuda': False, 'log_interval': 100, 'verbose': 1, 'filename': '../data/2020-07-01', 'seed': 2019, 'N_cv': 10, 'do_compute': True, 'save_model': True}
Overwriting tr

In [3]:
import torch
torch.cuda.is_available()

True

In [4]:
figname = '../figures/fig_methods_sup'

In [5]:
args

{'w': 28,
 'minibatch_size': 100,
 'train_batch_size': 50000,
 'test_batch_size': 10000,
 'noise_batch_size': 1000,
 'mean': 0.1307,
 'std': 0.3081,
 'what_offset_std': 15,
 'what_offset_max': 25,
 'N_pic': 128,
 'offset_std': 30,
 'offset_max': 34,
 'noise': 0.75,
 'contrast': 0.7,
 'sf_0': 0.1,
 'B_sf': 0.1,
 'do_mask': True,
 'N_theta': 6,
 'N_azimuth': 24,
 'N_eccentricity': 10,
 'N_phase': 2,
 'rho': 1.41,
 'bias_deconv': True,
 'p_dropout': 0.0,
 'dim1': 1000,
 'dim2': 1000,
 'lr': 0.005,
 'do_adam': True,
 'bn1_bn_momentum': 0.5,
 'bn2_bn_momentum': 0.5,
 'momentum': 0.3,
 'epochs': 60,
 'num_processes': 1,
 'no_cuda': False,
 'log_interval': 100,
 'verbose': 1,
 'filename': '../data/2020-07-01',
 'seed': 2019,
 'N_cv': 10,
 'do_compute': True,
 'save_model': True}

# Evaluating the accuracy of the classifier (What) wrt to spatial shifts

On commence par la fonction de base apprise de la librairie torch, cf https://raw.githubusercontent.com/pytorch/examples/master/mnist/main.py :

In [6]:
%whos

Variable        Type        Data/Info
-------------------------------------
args            EasyDict    {'w': 28, 'minibatch_size<...>True, 'save_model': True}
code            QRCode      QRCode(content=b'https://<...>version=6, mode='binary')
dpi_export      int         600
fig_width_pt    int         1024
figname         str         ../figures/fig_methods_sup
figwidth        float       14.169088141690882
inches_per_pt   float       0.013837000138370002
init            function    <function init at 0x7f7b8ef36d90>
np              module      <module 'numpy' from '/ho<...>kages/numpy/__init__.py'>
os              module      <module 'os' from '/usr/lib/python3.6/os.py'>
phi             float64     1.618033988749895
plt             module      <module 'matplotlib.pyplo<...>es/matplotlib/pyplot.py'>
ppi             float       72.27
pq              module      <module 'pyqrcode' from '<...>es/pyqrcode/__init__.py'>
torch           module      <module 'torch' from '/ho<...>kages/torch/__

In [7]:
from what import WhatNet
model = WhatNet()


On apprend une matrice de poids qui est fixée dans la suite et que nous allons utiliser pour la classification

In [8]:
model_path = "../data/MNIST_cnn.pt"

In [9]:
%ls -ltr {model_path}
#%rm -f {model_path}

ls: cannot access '../data/MNIST_cnn.pt': No such file or directory


In [10]:
import os
import torch
if os.path.isfile(model_path):
    print('Loading')
    model = torch.load(model_path)
    #model.load_state_dict(torch.load(model_path))
else:
    print('Learning')
    import time
    t0 = time.time()
    from what import main
    #%run what.py --epochs 10 --save-model
    main(args)
    print('Done in ', time.time() - t0, 'seconds')

Learning
use_cuda True
Training the What model
Train Epoch: 1/60 [0/60000 (0%)]	Loss: 2.302170
Train Epoch: 1/60 [10000/60000 (17%)]	Loss: 2.302622
Train Epoch: 1/60 [20000/60000 (33%)]	Loss: 2.297431
Train Epoch: 1/60 [30000/60000 (50%)]	Loss: 2.303493
Train Epoch: 1/60 [40000/60000 (67%)]	Loss: 2.292274
Train Epoch: 1/60 [50000/60000 (83%)]	Loss: 2.302521

Test set: Average loss: 0.0230, Accuracy: 1135/10000 (11%)

Train Epoch: 2/60 [0/60000 (0%)]	Loss: 2.291935
Train Epoch: 2/60 [10000/60000 (17%)]	Loss: 2.293992
Train Epoch: 2/60 [20000/60000 (33%)]	Loss: 2.296294
Train Epoch: 2/60 [30000/60000 (50%)]	Loss: 2.296739
Train Epoch: 2/60 [40000/60000 (67%)]	Loss: 2.306677
Train Epoch: 2/60 [50000/60000 (83%)]	Loss: 2.305442

Test set: Average loss: 0.0230, Accuracy: 1135/10000 (11%)

Train Epoch: 3/60 [0/60000 (0%)]	Loss: 2.294444
Train Epoch: 3/60 [10000/60000 (17%)]	Loss: 2.297145
Train Epoch: 3/60 [20000/60000 (33%)]	Loss: 2.302402
Train Epoch: 3/60 [30000/60000 (50%)]	Loss: 2.30661

Train Epoch: 22/60 [50000/60000 (83%)]	Loss: 2.302759

Test set: Average loss: 0.0230, Accuracy: 1135/10000 (11%)

Train Epoch: 23/60 [0/60000 (0%)]	Loss: 2.302987
Train Epoch: 23/60 [10000/60000 (17%)]	Loss: 2.312592
Train Epoch: 23/60 [20000/60000 (33%)]	Loss: 2.304318
Train Epoch: 23/60 [30000/60000 (50%)]	Loss: 2.307711
Train Epoch: 23/60 [40000/60000 (67%)]	Loss: 2.303343
Train Epoch: 23/60 [50000/60000 (83%)]	Loss: 2.296190

Test set: Average loss: 0.0230, Accuracy: 1135/10000 (11%)

Train Epoch: 24/60 [0/60000 (0%)]	Loss: 2.303475
Train Epoch: 24/60 [10000/60000 (17%)]	Loss: 2.304190
Train Epoch: 24/60 [20000/60000 (33%)]	Loss: 2.299060
Train Epoch: 24/60 [30000/60000 (50%)]	Loss: 2.307139
Train Epoch: 24/60 [40000/60000 (67%)]	Loss: 2.297291
Train Epoch: 24/60 [50000/60000 (83%)]	Loss: 2.299153

Test set: Average loss: 0.0230, Accuracy: 1135/10000 (11%)

Train Epoch: 25/60 [0/60000 (0%)]	Loss: 2.305438
Train Epoch: 25/60 [10000/60000 (17%)]	Loss: 2.293175
Train Epoch: 25/60 [20

Train Epoch: 44/60 [20000/60000 (33%)]	Loss: 2.302622
Train Epoch: 44/60 [30000/60000 (50%)]	Loss: 2.305707
Train Epoch: 44/60 [40000/60000 (67%)]	Loss: 2.307292
Train Epoch: 44/60 [50000/60000 (83%)]	Loss: 2.300454

Test set: Average loss: 0.0230, Accuracy: 1135/10000 (11%)

Train Epoch: 45/60 [0/60000 (0%)]	Loss: 2.310520
Train Epoch: 45/60 [10000/60000 (17%)]	Loss: 2.305330
Train Epoch: 45/60 [20000/60000 (33%)]	Loss: 2.304358
Train Epoch: 45/60 [30000/60000 (50%)]	Loss: 2.299617
Train Epoch: 45/60 [40000/60000 (67%)]	Loss: 2.300817
Train Epoch: 45/60 [50000/60000 (83%)]	Loss: 2.302171

Test set: Average loss: 0.0230, Accuracy: 1135/10000 (11%)

Train Epoch: 46/60 [0/60000 (0%)]	Loss: 2.312762
Train Epoch: 46/60 [10000/60000 (17%)]	Loss: 2.303669
Train Epoch: 46/60 [20000/60000 (33%)]	Loss: 2.304850
Train Epoch: 46/60 [30000/60000 (50%)]	Loss: 2.293423
Train Epoch: 46/60 [40000/60000 (67%)]	Loss: 2.298818
Train Epoch: 46/60 [50000/60000 (83%)]	Loss: 2.307284

Test set: Average loss:

In [11]:
args.no_cuda

False

In [12]:
from main import init
#args = init(filename='debug')
#args = init(filename='../data/2019-03-19_bis')
#args = init()
#args = init(filename='../data/2019-04-15_bis', verbose=1)
args = init(filename='../data/2020-07-01')

from where import Where
from what import WhatNet
where = Where(args)

use_cuda True
Retina vectorizing...
ok
success
Done vectorizing...
Inversing retina transform...
Done Inversing retina transform...


FileNotFoundError: [Errno 2] No such file or directory: '../data/MNIST_accuracy.npy'

In [ ]:

from main import init
#args = init(filename='debug')
args = init(filename='../data/2020-07-01')

from where import Where
from what import WhatNet
where = Where(args)

filename_train = args.filename + '_train.pt'
#%ls -lh {filename_train}
#%rm {filename_train}
#%rm  ../data/debug_train.pt

where.train(filename_train)



## Using the learned classifier in a standalone class

Maintenant qu'on a appris les points qui permet une classification d'à peu près 98 % on va utiliser le modèle fead-forward pour faire la classification.

## shifting the input images


Je vais maintenant générer des données en utilisant les données originales de MNIST translatées :

In [ ]:
from torchvision import datasets, transforms
i_shift, j_shift = 12, 17
N_pix = 28

In [ ]:
from display import Display
d = Display(args)
data, label = next(iter(d.loader_test))

In [ ]:
fig, ax = plt.subplots()
ax.imshow(data[0, 0, :, :].numpy())

In [ ]:
data = data.numpy()
data_translate = data.min() * np.ones((data.shape[0], 1, N_pix*3 - 2, N_pix*3 - 2))
print(data_translate.shape)
data_translate[:, :, (N_pix-i_shift):(2*N_pix-i_shift), (N_pix-j_shift):(2*N_pix-j_shift)] = data
fig, ax = plt.subplots()
ax.imshow(data_translate[0, 0, :, :])

In [ ]:
data_cropped = data_translate[:, :, (N_pix):(2*N_pix), (N_pix):(2*N_pix)]
fig, ax = plt.subplots()
ax.imshow(data_cropped[0, 0, :, :])

In [ ]:
np.arange(-N_pix+1, N_pix)

In [ ]:
def shift_data(data, i_shift, j_shift):
    N_pix = data.shape[-1]
    assert(N_pix == data.shape[-2])
    import numpy as np
    data_translate = data.min() * np.ones((data.shape[0], 1, N_pix*3 - 1, N_pix*3 - 1))
    data_translate[:, :, (N_pix+i_shift):(2*N_pix+i_shift), (N_pix+j_shift):(2*N_pix+j_shift)] = data
    data_cropped = data_translate[:, :, (N_pix):(2*N_pix), (N_pix):(2*N_pix)]
    return data_cropped

data_cropped = shift_data(data, i_shift = 12, j_shift = -12)
fig, ax = plt.subplots()
ax.imshow(data_cropped[0, 0, :, :])

## Using the learned classifier on the shifted data

On peut maintenant tester le classifieur sur les images Translatées en calculant la valeur de classification en  fonction de l'erreur de localisation

In [ ]:
def test_shift(test_loader, i_shift, j_shift, verbose=0):
    model.eval()
    test_loss = 0
    correct = 0
    for data, target in test_loader:
        data_cropped = shift_data(data, i_shift=i_shift, j_shift=j_shift)        
        data_cropped = torch.FloatTensor(data_cropped) #transforms.ToTensor()(data_cropped)
        data_cropped, target = Variable(data_cropped, volatile=True), Variable(target)
        output = model(data_cropped)
        test_loss += F.nll_loss(output, target, size_average=False).data[0] # sum up batch loss
        pred = output.data.max(1, keepdim=True)[1] # get the index of the max log-probability
        correct += pred.eq(target.data.view_as(pred)).cpu().sum()

    test_loss /= len(test_loader.dataset)
    if verbose: print('\nTest set: at ({}, {}), the  average loss is {:.4f}, Accuracy: {}/{} ({:.0f}%)'.format(
        i_shift, j_shift, test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))
    return correct / len(test_loader.dataset)


path = "../data/MNIST_accuracy.npy"

import os
if os.path.isfile(path):
    print('Loading accuracy')
    accuracy = np.load(path)
else:
    print('Computing accuracy')
    import time
    t0 = time.time()
    accuracy = np.zeros((2*N_pix-1, 2*N_pix-1))
    from tqdm import tqdm
    N_step = 1

    with tqdm(total=(2*N_pix-1)**2/N_step**2) as pbar:
        for i_shift in np.arange(-N_pix+1, N_pix, N_step):
            for j_shift in np.arange(-N_pix+1, N_pix, N_step):
                accuracy[i_shift+N_pix-1, j_shift+N_pix-1] = test_shift(test_loader, i_shift, j_shift)
                pbar.update()
    np.save(path, accuracy)
    print('Done in ', time.time() - t0, 'seconds')
    
print('accuracy=', accuracy)    

J'avoue que c'est un peu bourrin de calculer la classification sur les 128 × 128 pixels pour 1000 batch multiplié par 10 type d'entrées.... Mais bon on doit faire ça seulement une fois :-) (et sur CPU une classif = environ 300µs ...)

In [ ]:
fig, ax = plt.subplots(figsize=(6.5, 5.25))
cmap = ax.pcolor(np.arange(-N_pix+1, N_pix+1), np.arange(-N_pix+1, N_pix+1), accuracy, cmap=plt.plasma())
ax.axis('equal')
fig.colorbar(cmap)
fig.savefig('/tmp/panel_C.pdf', bbox_inches='tight', pad_inches=0.1)

Cette fonction de performance du classifieur  est calculée indépendamment de la forme spécifique du chiffre entre 0 et 9. Elle donne donc la carte de performance qu'on attend Au niveau de la classification/ On va pouvoir maintenant l'utiliser ceomm label pour apprendre de façon supervisée la correspondance entre la carte log-polaire obtenue depuis l'image brute et cette carte de performance.

# Retinotopic mapping

## Orientation invariant power encoding (colliculus??)

In [ ]:
from retina import Retina
r = Retina(args)

In [ ]:
print('r.colliculus.shape=', r.colliculus.shape)

In [ ]:
print('r.colliculus_vector.shape=', r.colliculus_vector.shape)

In [ ]:
print('r.colliculus_inverse.shape=', r.colliculus_inverse.shape)

In [ ]:
energy = (r.retina_transform**2).sum(axis=(0,3)) 
energy /= energy.sum(axis=-1)[:, :, None]
energy_vector = energy.reshape((args.N_azimuth*args.N_eccentricity, args.N_pic**2))
energy_plus = np.linalg.pinv(energy_vector)
FIG_WIDTH = 5 

fig, ax = plt.subplots(figsize=(figwidth/3, figwidth/3))
for i_orient in range(args.N_azimuth):
    for i_scale in range(args.N_eccentricity):
        env = np.sqrt(energy[i_orient, i_scale, :]**2.5).reshape((args.N_pic, args.N_pic))
        ax.contour(energy[i_orient, i_scale, :].reshape((args.N_pic, args.N_pic)), levels=[env.max()/2], lw=1,
                  colors=[plt.cm.rainbow(i_scale * 1.5/args.N_azimuth)])
#fig.suptitle('Tiling of visual space using energy', y=1.02)
ax.set_xlabel(r'$Y$')
ax.set_ylabel(r'$X$')
ax.axis('equal')
plt.tight_layout()
fig.savefig('/tmp/panel_B.pdf')

# summary


In [ ]:
import matplotlib.pyplot as plt
fig = plt.figure(figsize = (figwidth, figwidth/3.618))
ax_A = plt.subplot(1, 3, 1, projection='3d')

fig, ax_A = panel_A(fig, ax_A)
#data_retina = where.retina.retina(full[idx]['data_fullfield'])
#ax_A = where.retina.show(ax_A, where.retina.retina_invert(data_retina))
    

ax_B = plt.subplot(1, 3, 2)
for i_orient in range(0, args.N_azimuth, 2):
    for i_scale in range(1, args.N_eccentricity, 2):
        env = np.sqrt(energy[i_orient, i_scale, :]**2.5).reshape((args.N_pic, args.N_pic))
        ax_B.contour(energy[i_orient, i_scale, :].reshape((args.N_pic, args.N_pic)), levels=[env.max()/2], lw=.1,
                  colors=[plt.cm.rainbow(i_scale * 1.5/args.N_azimuth)])
ax_B.set_xlabel(r'$Y$')
ax_B.set_ylabel(r'$X$')
ax_B.axis('square')

ax_C = plt.subplot(1, 3, 3)
cmap = ax_C.pcolor(np.arange(-N_pix+1, N_pix+1), np.arange(-N_pix+1, N_pix+1), accuracy, cmap=plt.plasma())
ax_C.axis('square')
ax_C.set_xlabel(r'$\Delta Y$')
ax_C.set_ylabel(r'$\Delta X$')
fig.colorbar(cmap)


#for ax, text, x_offset, y_offset in [[ax_A, 'A', -.35, .95], [ax_B, 'B', -.35, .95], [ax_C, 'C', -.35, .95]]:
#    ax.text(x_offset, y_offset, '(' + text + ')', fontsize=24,
#              bbox={'facecolor':'white','alpha':1,'edgecolor':'none','pad':1},
#              ha='left', va='center', transform=ax.transAxes) 
plt.tight_layout()

fig.savefig(figname + '.pdf', bbox_inches='tight', pad_inches=0.1)

In [ ]:
import tikzmagic

In [ ]:
%load_ext tikzmagic

In [ ]:
%ls /tmp/panel*

In [ ]:
%%tikz -f pdf --save {figname}.pdf
\draw[white, fill=white] (0.\linewidth,0) rectangle (1.\linewidth, .382\linewidth) ;
\draw [anchor=north west] (.0\linewidth, .382\linewidth) node {\includegraphics[width=.33\linewidth]{/tmp/panel_A.pdf}};
\draw [anchor=north west] (.333\linewidth, .382\linewidth) node {\includegraphics[width=.31\linewidth]{/tmp/panel_B.pdf}};
\draw [anchor=north west] (.666\linewidth, .382\linewidth) node {\includegraphics[width=.35\linewidth]{/tmp/panel_C.pdf}};
\begin{scope}[font=\bf\sffamily\large]
\draw [anchor=west,fill=white] (.0\linewidth, .382\linewidth) node [above right=-3mm] {$\mathsf{D}$};
\draw [anchor=west,fill=white] (.36\linewidth, .382\linewidth) node [above right=-3mm] {$\mathsf{E}$};
\draw [anchor=west,fill=white] (.69\linewidth, .382\linewidth) node [above right=-3mm] {$\mathsf{F}$};
\end{scope}

# post-processing

In [ ]:
!convert  -density {dpi_export} {figname}.pdf {figname}.jpg
!convert  -density {dpi_export} {figname}.pdf {figname}.png
#!convert  -density {dpi_export} -resize 5400  -units pixelsperinch -flatten  -compress lzw  -depth 8 {fname}.pdf {fname}.tiff

In [ ]:
from IPython.display import Image
Image('{figname}.png'.format(figname=figname))

In [ ]:
!ls  -l {figname}*